In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
np.random.seed(333)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# !pip install gdown

In [3]:
# gdrive_folder_link="https://drive.google.com/drive/folders/1RpD6itBuWYCdFeTjVilt9CKPgDvqoitq?usp=sharing"

In [4]:
# import gdown
# gdown.download_folder(gdrive_folder_link, quiet=True)

In [5]:
import cv2
import time
import matplotlib.pyplot as plt

%matplotlib inline

In [6]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))

In [8]:
### num_epochs now = 2

class ModelConfig:
    EPOCHS = 10
    BATCH_SIZE = 4
    num_frames_per_second = 10
    SEQUENCE_SIZE = 16
    H = 128
    W = 128
    C = 3
    TO_GRAY = False
    overlapping = 0
    rootdir = "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos"
    TRAIN_SAMPLE_NPZ_DIRECTORY = "./npz_files"
    TEST_SAMPLE_NPZ_DIRECTORY = "./npz_files"
    types = {"Normal":0, "Abnormal":1}
    classes = {"Explosion":1, 'Burglary':2, 'Fighting':3, 'Assault':4, 'Arrest':5, 'Arson':6, 'Abuse':7}
    extension = "mp4"
    MODEL_WEIGHTS_DIRECTORY = "./model_weights"
    COMBINE_MODEL_PATH = "combined_model_weights.hdf5"
    GENERATOR_MODEL_PATH = "generator_model_weights.hdf5"
    DISCRIMINATOR_MODEL_PATH = "discriminator_model_weights.hdf5"
    CLASSIFIER_MODEL_PATH = "classifier_model_weights.hdf5"
    AUTOENCODER_MODEL_PATH = "autoencoder_model.hdf5"

In [9]:
os.mkdir(ModelConfig.TRAIN_SAMPLE_NPZ_DIRECTORY)
os.mkdir(ModelConfig.MODEL_WEIGHTS_DIRECTORY)

In [10]:
def get_video_times(cap):
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    print('fps = ' + str(fps))
    print('number of frames = ' + str(frame_count))
    print('duration (S) = ' + str(duration))
    minutes = int(duration / 60)
    seconds = duration % 60
    print('duration (M:S) = ' + str(minutes) + ':' + str(seconds))

In [11]:
def crop_image_black_background(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    hh, ww = thresh.shape
    thresh[hh:hh, 0:ww] = 0
    white = np.where(thresh == 255)
    
    if len(white[0]) <= 1 or len(white[1]) <= 1:
        return None
    
    xmin, ymin, xmax, ymax = np.min(white[1]), np.min(white[0]), np.max(white[1]), np.max(white[0])
    crop = img[ymin:ymax, xmin:xmax]
    
    if (crop.shape[0] < img.shape[0]/2) and (crop.shape[1] < img.shape[1]/2):
        return None
    return crop

In [12]:
def SaveVideo2Npz(file_path, npz_directory, resize=(ModelConfig.H, ModelConfig.W), 
                  num_target_frames=ModelConfig.SEQUENCE_SIZE, overlapping=0):
    global error_frame
    cap = cv2.VideoCapture(file_path)
    file_name = file_path.split('/')[-1]
    file_name = file_name.split('.')[0]
    fps = cap.get(cv2.CAP_PROP_FPS)
    #get_video_times(cap)
    len_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    segments_path_list = []
    if fps <= ModelConfig.num_frames_per_second:
        step = fps
    else:
        step = int(fps / ModelConfig.num_frames_per_second)
    try:
        frames = []
        num_sampled_video = 0
        frame_index = 0
        for i in range(0, len_frames):
            _, frame = cap.read()
            frame = crop_image_black_background(frame)
            if frame is None or (frame.shape[0] <= 0 or frame.shape[1] <= 0):
                continue
                
            if i % step == 0:
                frame = cv2.resize(frame, resize, interpolation=cv2.INTER_AREA)
                if ModelConfig.TO_GRAY:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    frame = frame.reshape((ModelConfig.H, ModelConfig.W, ModelConfig.C))
                else:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                frame = np.array(frame, dtype=np.float32)
                frame /= 255.0
                frames.append(frame)
                frame_index += 1

                if frame_index == num_target_frames:                    
                    segments_path = os.path.join(npz_directory, file_name + "_{}.npz".format(num_sampled_video))
                    num_sampled_video += 1
                    np.savez(segments_path, np.array(frames))
                    segments_path_list.append(segments_path)
                    if overlapping == 0:
                        frames.clear()
                        frame_index = 0
                    else:
                        for _ in range(0, overlapping):
                            frames.pop(0)
                            frame_index -= 1
    except Exception as e:
        print(e)
    finally:
        cap.release()

    return np.array(segments_path_list), fps

In [13]:
# toooooooooooooooooooooooooo much memory

skip_files = [
    "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Burglary/Burglary064_x264.mp4",
    "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Explosion/Explosion046_x264.mp4",
    "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Arrest/Arrest047_x264.mp4",
    "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Fighting/Fighting041_x264.mp4",
    "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Arson/Arson019_x264.mp4",
    "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_940_x264.mp4",
    "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Burglary/Burglary095_x264.mp4",
    "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_935_x264.mp4",
    "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/Normal_Videos_924_x264.mp4",
    "../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Arrest/Arrest049_x264.mp4"
]

In [14]:
# segments, _ = SaveVideo2Npz("../input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Arrest/Arrest049_x264.mp4",
#                             overlapping=ModelConfig.overlapping)

# print(segments)

In [15]:
def read_npz_file(file_path):
    dict_data = np.load(file_path)
    data = dict_data['arr_0']
    return data

In [16]:
# frames = read_npz_file(segments[3])
# plt.imshow(frames[1], cmap="gray")


In [17]:
def clear_npz_directory(directorty_path):
    for f in os.listdir(directorty_path):
        os.remove(os.path.join(directorty_path, f))
        
clear_npz_directory(ModelConfig.TRAIN_SAMPLE_NPZ_DIRECTORY)     

In [18]:
file_name_list = []
file_path_list = []
file_type_list = []
file_class_list = []
file_npy_path_list = []

In [19]:
def build_dataset_df():
    global file_name_list 
    global file_path_list
    global file_type_list 
    global file_class_list 
    global file_npy_path_list

    file_name_list.clear()
    file_path_list.clear()
    file_type_list.clear()
    file_class_list.clear()
    file_npy_path_list.clear()


    for root, subdirs, files in os.walk(ModelConfig.rootdir):
        for filename in files:
            if filename.split('.')[-1] != ModelConfig.extension:
                continue

            file_path = os.path.join(root, filename)
            file_name = filename.split('.')[0]
            
            if file_path in skip_files:
                continue
            
            file_name_list.append(file_name)
            file_path_list.append(file_path)
            file_class = file_path.split('/')[-2]
            #print(file_class)
            if file_class in ModelConfig.classes.keys():
                file_type_list.append(ModelConfig.types["Abnormal"])
                file_class_list.append(ModelConfig.classes[file_class])
            else:
                file_type_list.append(ModelConfig.types["Normal"])
                file_class_list.append(ModelConfig.types["Normal"])

            #npy_path = Save2Npy(file_path, file_name, Config.save_npy_dir)
            #file_npy_path_list.append(npy_path)

In [20]:
build_dataset_df()

In [21]:
len(file_path_list), len(file_name_list), len(file_type_list), len(file_class_list)#, len(file_npy_path_list)

In [22]:
#dataset_df = pd.DataFrame(list(zip(file_name_list, file_path_list, file_type_list, file_class_list, file_npy_path_list)),
#                          columns =['file_name', 'file_path', 'file_type', 'file_class', 'npy_file_path'])

dataset_df = pd.DataFrame(list(zip(file_name_list, file_path_list, file_type_list, file_class_list)),
                          columns =['file_name', 'file_path', 'file_type', 'file_class'])

In [23]:
dataset_df

In [24]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical


class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, dataset_df, X_col, y_col, directory, npz_directory, shuffle=True, data_augmentation=True):
        self.batch_size = 1
        self.dataset_df = dataset_df
        self.X_col = X_col
        self.y_col = y_col
        self.directory = directory
        self.npz_directory = npz_directory
        self.shuffle = shuffle
        self.data_aug = data_augmentation
        self.classes = np.unique(self.dataset_df[self.y_col])
        self.num_classes =  len(self.classes)
        self.X_path, self.Y_dict = self.search_data() 
        self.print_stats()
        return None
        
    def search_data(self):
        X_path = []
        Y_dict = {}
        one_hots = to_categorical(self.dataset_df[self.y_col], self.num_classes)
        for index in range(len(self.dataset_df)):
            X_path.append(self.dataset_df.at[index, self.X_col])
            Y_dict[X_path[-1]] = one_hots[index]
        return X_path, Y_dict
    
    def print_stats(self):
        self.n_files = len(self.X_path)
        self.indexes = np.arange(len(self.X_path))
        np.random.shuffle(self.indexes)
        print("Found {} files belonging to {} classes.".format(self.n_files,self.num_classes))
    
    def __len__(self):
        steps_per_epoch = np.ceil(len(self.X_path) / float(self.batch_size))
        return int(steps_per_epoch)

    def __getitem__(self, index):
        batch_indexs = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_path = [self.X_path[k] for k in batch_indexs]
        batch_x, batch_y = self.data_generation(batch_path)               
        return batch_x, batch_y
    
    def get_mini_batch(self, index):
        return self.__getitem__(index)

    def on_epoch_end(self):
        # shuffle the data at each end of epoch
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def data_generation(self, batch_path):
        batch_x = [self.load_data(x) for x in batch_path]
        batch_y = [self.Y_dict[x] for x in batch_path]

        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y
    
    def load_data(self, path):
        #print(path)
        segments, _  = SaveVideo2Npz(path, self.npz_directory, 
                                     resize=(ModelConfig.H, ModelConfig.W), 
                                     num_target_frames=ModelConfig.SEQUENCE_SIZE,
                                     overlapping=ModelConfig.overlapping)
        return segments

In [25]:
train_gen = CustomDataGen(dataset_df,
                           X_col="file_path",
                           y_col="file_class",
                           directory = ModelConfig.rootdir, 
                           npz_directory = ModelConfig.TRAIN_SAMPLE_NPZ_DIRECTORY,
                           shuffle=True)

In [26]:
train_gen.get_mini_batch(0)

In [27]:
import random 

n = random.randint(0, len(dataset_df))
randomlist = random.sample(range(0, len(dataset_df)), len(dataset_df)) 
val_size = 0.1
val_size = int(len(dataset_df) * val_size)
val_size = randomlist[:val_size]
validation_df = dataset_df.iloc[val_size].reset_index()

In [28]:
#validation_df

In [29]:
validation_gen = CustomDataGen(validation_df,
                               X_col="file_path",
                               y_col="file_class",
                               directory = ModelConfig.rootdir,
                               npz_directory=ModelConfig.TRAIN_SAMPLE_NPZ_DIRECTORY,
                               shuffle=True)

In [30]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Input, Activation, TimeDistributed,
                                     BatchNormalization, Conv3D, Conv2D, 
                                     LeakyReLU, Conv3DTranspose, Conv2DTranspose,
                                     ConvLSTM2D, LayerNormalization)
from tensorflow.keras.layers import MaxPool3D
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K


def encoder(X_input):
    # encoder    
    X = Conv3D(32, 3, padding='same')(X_input)
    X = BatchNormalization()(X)
    X = LeakyReLU()(X)
    X = MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid')(X)

    X = Conv3D(48, 3, padding='same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU()(X)
    X = MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid')(X)

    X = Conv3D(64, 3, padding='same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU()(X)
    X = MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid')(X)
    # current shape is 2x16x16x64
    X = Conv3D(64, 3, padding='same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU()(X)
    bottleneck = MaxPool3D(pool_size=(2, 2, 2), strides=(1, 1, 1), padding='same')(X)
    
    return bottleneck

    
def decoder(bottleneck):    
    X = Conv3DTranspose(48, 2, strides=(2, 2, 2), padding='valid', name="decoder_layer")(bottleneck)
    X = BatchNormalization()(X)
    X = LeakyReLU()(X)
    # current shape is 4x32x32x48
    X = Conv3DTranspose(32, 2, strides=(2, 2, 2), padding='valid')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU()(X)
    # current shape is 8x64x64x32
    X = Conv3DTranspose(32, 2, strides=(2, 2, 2), padding='valid')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU()(X)
    # current shape is 16x128x128x32
    X = Conv3D(ModelConfig.C, 3, strides=(1, 1, 1), padding='same')(X)
    X = Activation('sigmoid')(X)

    return X

def AutoEncoderModel(X_input):
    autoencoder = Model(X_input, decoder(encoder(X_input)), name='AutoEncoderModel')
    return autoencoder


def custom_loss(new, original):
    reconstruct_loss = K.mean(K.square(new-original))
    return reconstruct_loss

X_input = Input(shape=(ModelConfig.SEQUENCE_SIZE, ModelConfig.H,ModelConfig.W,ModelConfig.C))
autoEncoderModel = AutoEncoderModel(X_input)
opt = keras.optimizers.Adam(learning_rate=0.001)
autoEncoderModel.compile(
    loss=custom_loss, optimizer=opt, metrics=['accuracy'])
print(autoEncoderModel.summary())

In [31]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Conv3D,ConvLSTM2D, Flatten, Dense, Input,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam

def create_discriminator_model():    
    
    model = Sequential()
    
    model.add(Input((ModelConfig.SEQUENCE_SIZE,ModelConfig.H,ModelConfig.W,ModelConfig.C)))
    model.add(Conv3D(32, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid'))
    
    model.add(Conv3D(48, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid'))

    model.add(Conv3D(64, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid'))
    
    model.add(Conv3D(64, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(1, 1, 1), padding='same'))
    
    model.add(Conv3D(32, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(1, 1, 1), padding='same'))
    
    model.add(Conv3D(16, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(1, 1, 1), padding='same'))
    
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

# def DiscriminatorModel(X_input):
#     discriminator = Model(X_input, create_discriminator_model(X_input), name='Discriminator')
#     return discriminator

#X_input = Input(shape=(ModelConfig.H,ModelConfig.W,ModelConfig.SEQUENCE_SIZE,ModelConfig.C))
discriminator = create_discriminator_model()
opt = Adam(lr=0.001)
loss = BinaryCrossentropy()
discriminator.compile(loss=loss,
                      optimizer=opt,
                      metrics=['accuracy'])
print(discriminator.summary())

In [32]:
def visualize_result(autoencoder_model, gan_model, batch_num, X, epoch, img_seq_num=-1 ,image_idx=-1):
    mini_batch_size = X.shape[0]
    valid_y = np.array([1] * mini_batch_size)
    seq = autoencoder_model.predict(X)
    y_hat = gan_model.predict(X)
    
    result = []
    result.append(X[img_seq_num][image_idx])
    result.append(seq[img_seq_num][image_idx])
    result = np.array(result)
    print("Epoch: {}, Batch_number: {}".format(epoch, batch_num))
    print("y_hat:{}".format(round(y_hat[img_seq_num][0])))
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(18,20))
    for i, ax in enumerate(axs.flatten()):
        plt.sca(ax)
        if ModelConfig.TO_GRAY:
            plt.imshow(result[i].reshape(ModelConfig.H, ModelConfig.W), cmap="gray")
        else:
            plt.imshow(result[i])
        if i % 2 == 0:
            plt.title('Original Image: {}'.format(i+1))
        else:
            plt.title('Reconstructed Image: {}'.format(i+1))

    plt.show()

In [33]:
# import tensorflow as tf
import os
from tensorflow.keras.utils import Progbar

class GAN():
    def __init__(self):        
        self.image_shape=(ModelConfig.SEQUENCE_SIZE,ModelConfig.H, ModelConfig.W, ModelConfig.C)

        #learning_rate=0.0002
        #beta_1=0.5    

        opt1=Adam(learning_rate=1e-4, decay=1e-5, epsilon=1e-6)
        opt2=Adam(learning_rate=1e-4, decay=1e-5, epsilon=1e-6)
        opt3=Adam(learning_rate=1e-4, decay=1e-5, epsilon=1e-6)
        
        #Build and compile the discriminator
        self.discriminator=create_discriminator_model()
        self.discriminator.compile(loss=BinaryCrossentropy(),optimizer=opt1,metrics=['accuracy'])
        
        #Build and compile the generator
        X_input = Input(shape=(self.image_shape))
        self.generator = AutoEncoderModel(X_input)
        self.generator.compile(loss='mse',optimizer=opt2)
        
        #the generator takes a video as input and generates a modified video
        z = Input(shape=(self.image_shape))
        img = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(img)
        self.combined = Model(z, validity)
        self.combined.compile(loss=BinaryCrossentropy(), optimizer=opt3, metrics=['accuracy'])

    def re_init_combine(self, generator, discriminator):
        self.generator = generator
        self.discriminator = discriminator
        z = Input(shape=(self.image_shape))
        img = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(img)
        self.combined = Model(z, validity)
        opt3=Adam(lr=1e-4, decay=1e-5, epsilon=1e-6)
        self.combined.compile(loss=BinaryCrossentropy(), optimizer=opt3, metrics=['accuracy'])
        
    def load_weights(self):
        self.generator.load_weights(os.path.join(ModelConfig.MODEL_WEIGHTS_DIRECTORY, ModelConfig.GENERATOR_MODEL_PATH))
        self.discriminator.load_weights(os.path.join(ModelConfig.MODEL_WEIGHTS_DIRECTORY, ModelConfig.DISCRIMINATOR_MODEL_PATH))
        self.combined.load_weights(os.path.join(ModelConfig.MODEL_WEIGHTS_DIRECTORY, ModelConfig.COMBINE_MODEL_PATH))        

    def train_gan(self, train_gen):
        for epoch in range(ModelConfig.EPOCHS):
            d_loss_sum=np.zeros(2)
            g_loss_sum=np.zeros(2)
            reconstruct_loss_sum=0
            no_of_minibatches=0
            progress_bar = Progbar(target=len(train_gen))
            print("Epoch : ", epoch+1)
            for i in range(len(train_gen)):
                sample_reconstruct_loss=0
                sample_d_loss=np.zeros(2)
                sample_g_loss=np.zeros(2)
                X_sample, _ = train_gen.get_mini_batch(i)
                X_sample = X_sample[0]
                X_sample_size = X_sample.shape[0]                
                minibatch = None
                j = 0
                num_segments = 0
                while j < X_sample_size:
                    minibatch = []
                    mini_batch_size = min(ModelConfig.BATCH_SIZE, X_sample_size-j)
                    seg_indxes = list(range(j, j+mini_batch_size))
                    minibatch = np.array([read_npz_file(X_sample[index]) for index in seg_indxes])
                    j+=mini_batch_size                                         
                    gen_vids=self.generator.predict(minibatch)

                    #might have to combine these to improve batch norm
                    d_loss_real = self.discriminator.train_on_batch(minibatch, tf.ones((mini_batch_size,1)))
                    d_loss_fake = self.discriminator.train_on_batch(gen_vids, tf.zeros((mini_batch_size,1)))
                    d_loss=0.5*tf.math.add(d_loss_real,d_loss_fake)
                    # ---------------------
                    #  Train Generator
                    # ---------------------
                    # The generator wants the discriminator to label the generated samples as valid (ones)
                    valid_y = tf.ones((mini_batch_size,1))
                    # Train the generator
                    g_loss, _ = self.combined.train_on_batch(minibatch,valid_y)

                    reconstruct_loss=self.generator.train_on_batch(minibatch,minibatch)

                    sample_d_loss += d_loss
                    sample_g_loss += g_loss
                    sample_reconstruct_loss += reconstruct_loss
                    num_segments += 1
                clear_npz_directory(ModelConfig.TRAIN_SAMPLE_NPZ_DIRECTORY)
                
                if i % 10 == 0:
                    visualize_result(self.generator, self.combined, i, minibatch, epoch+1, img_seq_num=-1 ,image_idx=-1)
                    self.generator.save_weights(os.path.join(ModelConfig.MODEL_WEIGHTS_DIRECTORY, ModelConfig.GENERATOR_MODEL_PATH))
                    self.discriminator.save_weights(os.path.join(ModelConfig.MODEL_WEIGHTS_DIRECTORY, ModelConfig.DISCRIMINATOR_MODEL_PATH))
                    self.combined.save_weights(os.path.join(ModelConfig.MODEL_WEIGHTS_DIRECTORY, ModelConfig.COMBINE_MODEL_PATH))
                    
                g_loss_sum += (sample_g_loss/num_segments)
                d_loss_sum += (sample_d_loss/num_segments)
                reconstruct_loss_sum += (sample_reconstruct_loss/num_segments)
                progress_bar.update(i+1, 
                                    values=[('d_loss', (sample_d_loss[0]/num_segments)),
                                            ('g_loss', (sample_g_loss[0]/num_segments)),
                                            ('rec_loss', (sample_reconstruct_loss/num_segments))])
                print()

            g_loss=g_loss_sum/len(train_gen)
            d_loss=d_loss_sum/len(train_gen)
            reconstruct_loss=reconstruct_loss_sum/len(train_gen)

            print("(^|^)  ('|')")
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f, accuracy %.2f%% from which %f is combined loss and %f is reconstruction loss]" % 
                  (epoch+1, 
                   d_loss[0], 
                   100*d_loss[1], 
                   g_loss[0]+reconstruct_loss,
                   g_loss[1]*100,
                   g_loss[0],
                   reconstruct_loss))
        
            self.generator.save_weights(os.path.join(ModelConfig.MODEL_WEIGHTS_DIRECTORY, ModelConfig.GENERATOR_MODEL_PATH))
            self.discriminator.save_weights(os.path.join(ModelConfig.MODEL_WEIGHTS_DIRECTORY, ModelConfig.DISCRIMINATOR_MODEL_PATH))
            self.combined.save_weights(os.path.join(ModelConfig.MODEL_WEIGHTS_DIRECTORY, ModelConfig.COMBINE_MODEL_PATH))
            train_gen.on_epoch_end()

In [34]:
gan = GAN()

In [35]:
print(gan.generator.summary())
print(gan.discriminator.summary())
print(gan.combined.summary())

In [36]:
# gan.load_weights()

In [ ]:
with tf.device(device_name):
    gan = GAN()
#     gan.load_weights()
    gan.train_gan(train_gen)